Exemplo de Auto Machine Learning:

Vamos construir um modelo de AML e um de Random Forest para compara-los

In [2]:
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators import H2ORandomForestEstimator
import pandas as pd

In [3]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,23 mins 55 secs
H2O_cluster_timezone:,America/Sao_Paulo
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.1.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_started_from_R_Giovanni_hsp778
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,1.871 Gb
H2O_cluster_total_cores:,4
H2O_cluster_allowed_cores:,4
H2O_cluster_status:,"locked, healthy"


In [4]:
#Importa dados 
imp = pd.read_csv('Churn_treino.csv', sep=";")
imp.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0,1,1,1,10134888,1
1,608,Spain,Female,41,1,8380786,1,0,1,11254258,0
2,502,France,Female,42,8,1596608,3,1,0,11393157,1
3,699,France,Female,39,1,0,2,0,0,9382663,0
4,850,Spain,Female,43,2,12551082,1,1,1,790841,0


In [5]:
# divide em treino e teste
imp = h2o.H2OFrame(imp)
treino,teste = imp.split_frame(ratios=[.7])
    # ratios = .7 = 70% dos dados são de treino

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [6]:
#Transforma a variável dependente em fator
treino["Exited"] = treino["Exited"].asfactor()
teste["Exited"] = teste["Exited"].asfactor()

Modelo de AML:

In [7]:
modelo_automl = H2OAutoML(max_runtime_secs=180, sort_metric = 'AUC')
    # max_runtime_secs = tempo máximo de busca em uma métrica, 180s
    # sort_metric = métrica de avaliação
modelo_automl.train( y="Exited", training_frame=treino)
    # Treinamento, y = classe

AutoML progress: |█
00:52:25.312: AutoML: XGBoost is not available; skipping it.

███████████████████████████████████████████████████████| 100%


Modelo Random Forest:

In [8]:
model_floresta = H2ORandomForestEstimator(nfolds=5)
model_floresta.train( y="Exited", training_frame=treino)

drf Model Build progress: |███████████████████████████████████████████████| 100%


Ranqueando os melhores AML:

In [9]:
ranking = modelo_automl.leaderboard
ranking = ranking.as_data_frame()
ranking

,model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
0,StackedEnsemble_AllModels_AutoML_20200919_005225,0.858196,0.348946,0.697854,0.236597,0.324627,0.105383
1,GBM_grid__1_AutoML_20200919_005225_model_10,0.858052,0.345741,0.692713,0.239690,0.324279,0.105157
2,GBM_2_AutoML_20200919_005225,0.856561,0.345008,0.695031,0.234922,0.323902,0.104912
3,GBM_grid__1_AutoML_20200919_005225_model_11,0.856434,0.346505,0.694639,0.239051,0.324547,0.105331
4,StackedEnsemble_BestOfFamily_AutoML_20200919_0...,0.856279,0.351271,0.690553,0.232004,0.325915,0.106221
5,GBM_grid__1_AutoML_20200919_005225_model_12,0.854963,0.347347,0.692014,0.230516,0.324925,0.105576
6,GBM_5_AutoML_20200919_005225,0.854642,0.347896,0.689220,0.234714,0.325563,0.105991
7,GBM_1_AutoML_20200919_005225,0.854515,0.345845,0.694180,0.235481,0.323843,0.104874
8,GBM_grid__1_AutoML_20200919_005225_model_4,0.851986,0.352457,0.682133,0.246994,0.327764,0.107429
9,GBM_grid__1_AutoML_20200919_005225_model_8,0.849735,0.353569,0.682387,0.236662,0.328123,0.107665


Foram criados 28 modelos
O melhor foi: StackedEnsemble_AllModels_AutoML_20200919_005225
Que teve o AUC de 0.858

Performance do Random Forest:

In [10]:
performance = model_floresta.model_performance(test_data=teste)
performance


ModelMetricsBinomial: drf
** Reported on test data. **

MSE: 0.10206028895736116
RMSE: 0.3194687605343614
LogLoss: 0.3785837843553372
Mean Per-Class Error: 0.21811967727476467
AUC: 0.8570305945361975
AUCPR: 0.6854405048900383
Gini: 0.7140611890723949

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.32999999999999996: 


,,0,1,Error,Rate
0,0,2092.0,300.0,0.1254,(300.0/2392.0)
1,1,189.0,393.0,0.3247,(189.0/582.0)
2,Total,2281.0,693.0,0.1644,(489.0/2974.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,0.33000,0.616471,151.0
1,max f2,0.19000,0.687609,235.0
2,max f0point5,0.56475,0.683003,70.0
3,max accuracy,0.56475,0.868191,70.0
4,max precision,1.00000,1.000000,0.0
5,max recall,0.00000,1.000000,399.0
6,max specificity,1.00000,1.000000,0.0
7,max absolute_mcc,0.52000,0.535441,80.0
8,max min_per_class_accuracy,0.24000,0.778351,208.0
9,max mean_per_class_accuracy,0.29000,0.781880,176.0



Gains/Lift Table: Avg response rate: 19,57 %, avg score: 21,74 %


,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain,kolmogorov_smirnov
0,1,0.011432,0.9600,4.809379,4.809379,0.941176,0.970588,0.941176,0.970588,0.054983,0.054983,380.937942,380.937942,0.054147
1,2,0.025219,0.9200,4.860699,4.837434,0.951220,0.928740,0.946667,0.947711,0.067010,0.121993,386.069902,383.743414,0.120321
2,3,0.031607,0.9000,5.109966,4.892520,1.000000,0.900877,0.957447,0.938245,0.032646,0.154639,410.996564,389.252029,0.152967
3,4,0.040013,0.8408,4.905567,4.895261,0.960000,0.868350,0.957983,0.923561,0.041237,0.195876,390.556701,389.526120,0.193786
4,5,0.051446,0.8000,4.208207,4.742582,0.823529,0.820902,0.928105,0.900748,0.048110,0.243986,320.820699,374.258249,0.239388
5,6,0.100202,0.6135,3.418391,4.098261,0.668966,0.694436,0.802013,0.800361,0.166667,0.410653,241.839080,309.826103,0.385987
6,7,0.152320,0.4800,2.472564,3.542007,0.483871,0.532901,0.693157,0.708846,0.128866,0.539519,147.256402,254.200709,0.481409
7,8,0.208473,0.3800,1.499331,2.991802,0.293413,0.418133,0.585484,0.630541,0.084192,0.623711,49.933123,199.180246,0.516270
8,9,0.306658,0.2600,1.382491,2.476540,0.270548,0.305555,0.484649,0.526488,0.135739,0.759450,38.249070,147.654036,0.562962
9,10,0.402152,0.1800,0.719713,2.059367,0.140845,0.210100,0.403010,0.451359,0.068729,0.828179,-28.028653,105.936742,0.529684


O Random Forest teve o AUC de AUC: 0.8570305945361975
O AML foi 0.858, levemente melhor 

Prevendo com AML:

In [11]:
teste = pd.read_csv('Churn_prever.csv', sep=";")
teste = h2o.H2OFrame(teste)
prever = modelo_automl.leader.predict(teste) # leader = usa o melhor modelo para previsão
prever

Parse progress: |█████████████████████████████████████████████████████████| 100%
stackedensemble prediction progress: |████████████████████████████████████| 100%


predict,p0,p1
0,0.920235,0.0797651
0,0.926045,0.0739549
1,0.0394558,0.960544
0,0.943532,0.0564679
0,0.931241,0.0687586
0,0.906473,0.0935271
0,0.848754,0.151246
0,0.873545,0.126455
0,0.91723,0.0827702
